#### **Análise de negócios**

#### **Observações**

#### **Conteúdo - Bases e Notebook da aula**

Github:  

https://github.com/FIAP/Pos_Tech_DTAT/tree/Analise-de-Negocios/Analise%20de%20Negocios  

Série Histórica de Preços de Combustíveis e de GLP:  

https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis

#### **Importação de pacotes e bibliotecas**

In [ ]:
# Importar biblioteca completa
import pandas as pd
import os
import requests


from bs4 import BeautifulSoup
import sys

# Importar algo especifico de uma biblioteca
from dotenv import load_dotenv

#### **Funções (def)**

In [ ]:
def encontrar_links(soup):
    """
    Encontra dinamicamente os links de download na seção 
    'Combustíveis automotivos'.
    """
    
    # Encontra o cabeçalho <h3> que contém o texto "Combustíveis automotivos"
    # Usamos uma função lambda para ser mais robusto contra tags aninhadas (como <strong>)
    heading = soup.find(lambda tag: tag.name == 'h3' and 'Combustíveis automotivos' in tag.get_text())
    
    links_para_baixar = []
    
    if not heading:
        print("Erro: Não foi possível encontrar a seção 'Combustíveis automotivos' no HTML da página.")
        return links_para_baixar

    # A lista <ul> com os links é o próximo "irmão" (sibling) da tag <h3>
    ul_tag = heading.find_next_sibling('ul')
    
    if not ul_tag:
        print("Erro: Não foi possível encontrar a lista <ul> após o cabeçalho.")
        return links_para_baixar

    # Encontra todas as tags <a> (links) dentro desta lista <ul>
    a_tags = ul_tag.find_all('a')
    
    for a_tag in a_tags:
        url = a_tag.get('href')
        if url:
            # Os links no site já são absolutos (começam com 'https://'),
            # então não precisamos nos preocupar em juntar com a URL base.
            links_para_baixar.append(url)
            
    return links_para_baixar

In [ ]:
def baixar_arquivo(url, pasta_destino):
    """
    Baixa um único arquivo de uma URL para a pasta de destino.
    """
    # Pega o nome do arquivo da URL (ex: 'ca-2025-01.zip')
    nome_arquivo = os.path.basename(url)
    caminho_local = os.path.join(pasta_destino, nome_arquivo)
    
    try:
        print(f"Baixando {nome_arquivo}...")
        
        # Usa stream=True para baixar arquivos grandes sem carregar tudo na memória
        with requests.get(url, stream=True) as r:
            # Verifica se a requisição foi bem-sucedida
            r.raise_for_status()
            
            # Abre o arquivo local em modo 'write-binary' ('wb')
            with open(caminho_local, 'wb') as f:
                # Escreve o arquivo em "pedaços" (chunks) para eficiência de memória
                for chunk in r.iter_content(chunk_size=8192): 
                    f.write(chunk)
                    
        print(f"Concluído: {nome_arquivo}\n")
        
    except requests.exceptions.RequestException as e:
        print(f"Erro ao baixar {url}: {e}")

#### **Credenciais**

In [ ]:
load_dotenv()

# Credenciais do PostgreSQL
usuario_pg = os.getenv("POSTGRES_USER")
senha_pg = os.getenv("POSTGRES_PASSWORD")
host_pg = os.getenv("POSTGRES_HOST")
porta_pg = os.getenv("POSTGRES_PORT")
banco_pg = os.getenv("POSTGRES_DB")

#### **Aula 1 - Processos e formas de análise**

#### **Aula 2 - Ligação com bancos de dados**

In [ ]:
# URL da página de onde vamos extrair os links
PAGE_URL = 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis'

# Pasta para onde os arquivos serão baixados
DOWNLOAD_DIR = 'arquivos_combustiveis_automotivos'

# 1. Criar a pasta de download se ela não existir
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

# 2. Baixar o HTML da página da ANP
print(f"Acessando a página: {PAGE_URL}")
try:
    response = requests.get(PAGE_URL)
    # Verifica se a requisição foi bem-sucedida (código 200)
    response.raise_for_status()
    conteudo_html = response.text
    
except requests.exceptions.RequestException as e:
    print(f"Erro fatal ao acessar a página da ANP: {e}")
    print("Verifique sua conexão com a internet ou se a URL da ANP mudou")
    sys.exit(1) # Termina o script se não conseguir acessar a página

# 3. Analisar (parse) o HTML
soup = BeautifulSoup(conteudo_html, 'html.parser')

# 4. Encontrar os links
links = encontrar_links(soup)

if not links:
    print("Nenhum link encontrado para baixar")

else:
    print(f"Encontrados {len(links)} arquivos para baixar na seção 'Combustíveis automotivos'")
    print()

    # 5. Baixar cada arquivo
    for url in links:
        baixar_arquivo(url, DOWNLOAD_DIR)
        
    print()
    print("Download de todos os arquivos concluído")
    print(f"Os arquivos estão salvos em: {os.path.abspath(DOWNLOAD_DIR)}")
